In [28]:
import xml.etree.ElementTree as ET
import json
# 解析XML文件
"""
转化下数据
"""

def build_dataset(dataset, split):
    tree = ET.parse(dataset + '/' + split + "_" + dataset + '.xml')
    root = tree.getroot()
    data = []
    for i in root.findall('sentence'):
        single_data = {'text': i.find('text').text, "id": i.get('id')}
        data.append(single_data)
        if i.find('aspectTerms'):
            for e in i.find('aspectTerms').findall('aspectTerm'):
                term = e.get('term')
                polarity = e.get('polarity') # negative/positive/neutral/conflict
                implicit = e.get('implicit_sentiment', "####") # False/True
                opinion = e.get('opinion_words', '####') # False/True
                start = e.get('from')
                end = e.get('to')
                if "aspects" in single_data:
                    single_data['aspects'].append({'term': term, "polarity": polarity, "implicit": implicit, "from_to": [start, end], "opinion": opinion})
                else:
                    single_data['aspects'] = [{'term': term, "polarity": polarity, "implicit": implicit, "from_to": [start, end], "opinion": opinion}]
        if single_data not in data:
            data.append(single_data)

    # tmp_data = []
    # for i in data:
    #     if 'aspects' in i:
    #         answers = []
    #         aspects = []
    #         text = i['text']
    #         for j in i['aspects']:
    #             aspects.append(j['term'])
    #             answers.append(j['polarity'])
    # #             user_prompt = """<|im_start|>user
    # # Please analyze the sentiment polarity of aspect {} in sentence "{}"<|im_end|>""".format(', '.join(aspects), ', '.join(answers))
    #         user_prompt = "Analyze the sentiment polarity of " + str(len(aspects))
    #         if len(aspects) > 1:
    #             user_prompt += " aspects “{}” in the comment “{}”".format(', '.join(aspects), text)
    #         else:
    #             user_prompt += " aspect “{}” in the comment “{}”".format(', '.join(aspects), text)
    #         tmp_data.append({"question":user_prompt, "answer":', '.join(answers), "original_data":i})
    tmp_data = []
    for i in data:
        if 'aspects' in i:
            text = i['text']
            for j in i['aspects']:
                user_prompt = "Analyze the sentiment polarity of aspects “{}” in the comment “{}”".format(j['term'], text)
                tmp_data.append({"question":user_prompt, "answer":j['polarity'], "original_data":i})
    print(len(tmp_data))
    with open(dataset+"_"+split+".json", "w+", encoding='utf-8') as f:
        json.dump(tmp_data, f, ensure_ascii=False, indent=4)

datasets = ["laptops", "restaurants"]
splits = ["train", "test"]
# datasets = ["laptops"]
# splits = ["train"]
for d in datasets:
    for s in splits:
        build_dataset(d, s)

2358
654
3693
1134


In [25]:
data

[{'text': 'I charge it at night and skip taking the cord with me because of the good battery life.',
  'id': '2339',
  'aspects': [{'term': 'cord',
    'polarity': 'neutral',
    'implicit': 'True',
    'from_to': ['41', '45'],
    'opinion': '####'},
   {'term': 'battery life',
    'polarity': 'positive',
    'implicit': 'False',
    'from_to': ['74', '86'],
    'opinion': 'good'}]},
 {'text': 'I bought a HP Pavilion DV4-1222nr laptop and have had so many problems with the computer.',
  'id': '812'},
 {'text': 'The tech guy then said the service center does not do 1-to-1 exchange and I have to direct my concern to the "sales" team, which is the retail shop which I bought my netbook from.',
  'id': '1316',
  'aspects': [{'term': 'service center',
    'polarity': 'negative',
    'implicit': 'True',
    'from_to': ['27', '41'],
    'opinion': '####'},
   {'term': '"sales" team',
    'polarity': 'negative',
    'implicit': 'True',
    'from_to': ['109', '121'],
    'opinion': '####'},
   

In [2]:
import json
with open("laptops_train.json", "r+", encoding='utf-8') as f:
        train_data = json.load(f)
with open("laptops_test.json", "r+", encoding='utf-8') as f:
        test_data = json.load(f)
with open("laptops_train_test.json", "w+", encoding='utf-8') as f:
    tmp = train_data+test_data
    print(len(tmp))
    json.dump(train_data+test_data, f, ensure_ascii=False, indent=4)

6024


In [1]:
a = [1,2,3]
b = [4,5,6]
a+b

[1, 2, 3, 4, 5, 6]

In [25]:
system_prompt = """You are a sentiment analysis assistant, performing sentiment analysis on a given laptop product review and corresponding aspects. There are only four emotion categories: positive, negative, conflict, and neutral. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <reasoning> </reasoning> and <answer> </answer> tags, respectively, i.e., <reasoning> reasoning process here </reasoning> <answer> answer here </answer>. For example, analyze the sentiment polarity of 4 aspects “battery, screen, price and warranty” in sentence "Good battery, bad screen, the price is expensive but worth it, not sure about the warranty". <reasoning> Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in four aspects. The sentence expresses distinct opinions about different aspects. The phrase “Good battery” clearly conveys a positive sentiment toward the battery, using the adjective “good” without any qualifiers. In contrast, “bad screen” directly reflects a negative sentiment about the screen, as “bad” is an unambiguous negative descriptor. The statement “the price is expensive but worth it” presents a conflicted view of the price: while “expensive” suggests a negative perception, “worth it” introduces a compensating positive judgment, indicating mixed feelings. Finally, “not sure about the warranty” reveals uncertainty rather than any clear opinion, making the sentiment toward the warranty neutral. </reasoning> <answer> positive, negative, conflict, neutral </answer>"""

system_prompt = """You are a sentiment analysis assistant. Given a laptop product review and a list of aspects, your task is to determine the sentiment polarity for each aspect. The possible sentiment categories are: positive, negative, conflict, and neutral.

You must first explain your reasoning and then provide the final sentiment labels. Wrap your explanation inside <reasoning> tags and the final output inside <answer> tags. The format must be:

<reasoning> your reasoning here </reasoning>
<answer> label1, label2, ... </answer>

Example:
Sentence: "Good battery, bad screen, the price is expensive but worth it, not sure about the warranty"
Aspects: battery, screen, price, warranty
<reasoning>Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in 4 aspects. The phrase “Good battery” expresses a clearly positive opinion toward the battery. “bad screen” reflects a negative sentiment about the screen. “the price is expensive but worth it” indicates a mix of negative and positive cues, thus a conflicted sentiment. “not sure about the warranty” shows uncertainty, so the sentiment is neutral. </reasoning>
<answer>positive, negative, conflict, neutral</answer>"""

In [33]:
import re

text = """1235436<reasoning>Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in 4 aspects. The phrase “Good battery” expresses a clearly positive opinion toward the battery. “bad screen” reflects a negative sentiment about the screen. “the price is expensive but worth it” indicates a mix of negative and positive cues, thus a conflicted sentiment. “not sure about the warranty” shows uncertainty, so the sentiment is neutral. </reasoning>
<answer>positive, negative, conflict, neutral</answer>547239875"""

# 正则模式
pattern = r"<reasoning>((?:(?!<reasoning>|</reasoning>).)*?)</reasoning>\n<answer>((?:(?!<answer>|</answer>).)*?)</answer>"

# 查找第一个匹配
match = re.search(pattern, text, re.DOTALL)

# 输出结果
if match:
    print("✅ 提取成功：\n")
    print(match.group())
else:
    print("❌ 没有找到符合格式的内容")

✅ 提取成功：

<reasoning>Let’s analyze it step by step. Users need to analyze the sentiment polarity of words in 4 aspects. The phrase “Good battery” expresses a clearly positive opinion toward the battery. “bad screen” reflects a negative sentiment about the screen. “the price is expensive but worth it” indicates a mix of negative and positive cues, thus a conflicted sentiment. “not sure about the warranty” shows uncertainty, so the sentiment is neutral. </reasoning>
<answer>positive, negative, conflict, neutral</answer>


In [36]:
None in ["a"]

False

In [ ]:
请阅读以下评论，并结合给定的方面词和情感极性，进行逐个方面的情感分析，推理过程应自然流畅，体现出模型在理解语义后作出的判断，而非列出参考答案。

【评论】：
I am pleased with the fast log on, speedy WiFi connection and the long battery life (>6 hrs).

【方面词及其情感】：
log on：positive
WiFi connection: positive
battery life: positive

请你结合文本内容，对每个方面进行推理分析，写出你是如何判断出其情感极性的。推理可以适当引用评论中的具体句子，但不要照搬原文，要用自己的话组织语言，使推理过程看起来像是你自己思考后的结果。
推理过程必须用英文表达。
如果包含多个方面词，只需要在一段话中返回全部推理。
只需要返回推理过程，不需要返回任何解释和说明，以json格式返回。

【输出格式】：
{“reasoning”：reasoning}  -> {str:str}

In [5]:
import json
from collections import Counter
# 打开文件并读取JSON数据
max_len = 0
with open("laptops_train_test.json", 'r', encoding='utf-8') as f:
    data = json.load(f)
for i in data:
    # if len(i["question"]) > max_len:
    #     max_len = len(i["question"])
    if len(i["question"]) == 540:
        print(i)
print(max_len)

{'question': "Analyze the sentiment polarity of aspects “Windows update” in the comment “In\xa0November my computer messed up entirely and wouldn't power on after intalling a Windows update, I had to have my HD flashed and lost EVERYTHING on it, including my school assignments and irriplaceable pictures that were only in digital format and several other things, when this update was installed for some reason I was unable to roll back the drivers and everything to an earlier working condition because when the update was installed it deleted my history.”", 'answer': 'neutral', 'original_data': {'text': "In\xa0November my computer messed up entirely and wouldn't power on after intalling a Windows update, I had to have my HD flashed and lost EVERYTHING on it, including my school assignments and irriplaceable pictures that were only in digital format and several other things, when this update was installed for some reason I was unable to roll back the drivers and everything to an earlier wor